<a href="https://colab.research.google.com/github/maikCyphlock/Mylab/blob/main/whisper-faslk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/sanchit-gandhi/whisper-jax.git

!pip install flask-ngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sanchit-gandhi/whisper-jax.git to /tmp/pip-req-build-pbtn1ww6
  Running command git clone --filter=blob:none --quiet https://github.com/sanchit-gandhi/whisper-jax.git /tmp/pip-req-build-pbtn1ww6
  Resolved https://github.com/sanchit-gandhi/whisper-jax.git to commit 68b658ffe7bf5ab4ea01a78e3b1a9c207db3e144
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Descarga el archivo
!pip install pyngrok


# Descarga el archivo
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# Descomprime el archivo en la misma ruta
!tar -xzf ngrok-v3-stable-linux-amd64.tgz

# Elimina el archivo comprimido
!rm ngrok-v3-stable-linux-amd64.tgz

!/content/ngrok config add-authtoken 2D91FFQtqYkkGgUnospalrROWRd_2aG5upiFWGmQCZqKLqpeG


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-04-25 15:40:06--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8379857 (8.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   7.99M  45.5MB/s    in 0.2s    

2023-04-25 15:40:07 (45.5 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8379857/8379857]

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from requests.exceptions import Timeout
from flask import Flask, jsonify, request
import requests
from whisper_jax import FlaxWhisperPipline
from pyngrok import ngrok, conf



# Construct the API client
# client = ngrok.Client("2D91FFQtqYkkGgUnospalrROWRd_2aG5upiFWGmQCZqKLqpeG")
app = Flask(__name__)
conf.API_KEY = "2D91FFQtqYkkGgUnospalrROWRd_2aG5upiFWGmQCZqKLqpeG"
# instantiate pipeline
pipeline = FlaxWhisperPipline("openai/whisper-tiny")

# define route to handle POST requests
@app.route('/transcribe', methods=['POST'])
def transcribe():
    # check if request has an audio URL
    if 'url' not in request.get_json():
        return jsonify({'error': 'Missing URL parameter'}), 400

    # download audio file from URL
    url = request.get_json()['url']
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return jsonify({'error': str(e)}), 400

    # save audio file to disk
    with open('audio.mp3', 'wb') as f:
        for chunk in response.iter_content(chunk_size=512):
            if chunk:
                f.write(chunk)

    # transcribe audio file using pipeline
    text = pipeline('./audio.mp3', task="transcribe",return_timestamps=True)

    return jsonify({'text': text['text']}), 200

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"Ngrok URL: {public_url}")
    app.run(port=5000)


Ngrok URL: https://46fc-34-86-116-81.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Apr/2023 16:02:28] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2023 16:03:00] "POST /transcribe HTTP/1.1" 200 -
